In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.max_rows = 6

### groupby를 활용하여 그룹별 전처리

In [3]:
tips = sns.load_dataset('tips')

#### 1. agg : 각 그룹마다의 하나의 함수값(평균, 분산 등)을 계산
    -  하나의 value를 return

In [4]:
def min_max(group):
    return group.max() - group.min()

In [5]:
tips.groupby('sex').agg(min_max)

,total_bill,tip,size
sex,,,
Male,43.56,9.0,5
Female,41.23,5.5,5


#### 각 컬럼별로 다른 함수로 계산 가능

In [6]:
tips.groupby('sex').agg({'sex':len,'tip':np.mean, 'size':np.max})

,sex,tip,size
sex,,,
Male,157,3.089618,6
Female,87,2.833448,6


### 2. filter
    - 단순 DF.filter 함수와 사용방식이 다르다.
    - 각 row마다 자신이 속한 그룹의 조건식을 확인하고 T/F 반환

In [18]:
tips.groupby('sex')['size'].mean()

sex
Male      2.630573
Female    2.459770
Name: size, dtype: float64

In [22]:
# female은 size의 평균이 2.5보다 작기 때문에 male만 추출되는 결과
test = tips.groupby('sex').filter(lambda group: group['size'].mean() >=2.5)
test

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [23]:
print(test.sex.unique())

[Male]
Categories (1, object): [Male]


### 3. transform : apply처럼 모든 값에 대하여 계산
    -  해당 value가 속하는 그룹의 결과(mean, sum 등)를 활용할 수 있다는 것이 다름

In [8]:
tips[:2]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [9]:
def zscore(group):
    return (group - group.mean()) / group.std()

In [10]:
# 남자는 남자의 평균과 표준편차, 여자는 여자의 평균과 표준편차를 활용하여 계산한다.
tips.groupby('sex').tip.transform(zscore)

0     -1.572623
1     -0.960054
2      0.275590
         ...   
241   -0.731728
242   -0.899615
243    0.143642
Name: tip, Length: 244, dtype: float64

In [11]:
# 그룹별로 잘 계산이 되는 것 검증
by_total = round((tips.tip[0] - tips.tip.mean())/tips.tip.std(),5)
by_female = round((tips.tip[0] - tips.groupby('sex').tip.mean()['Female'])/tips.groupby('sex').tip.std()['Female'],5)
by_male = round((tips.tip[0] - tips.groupby('sex').tip.mean()['Male'])/tips.groupby('sex').tip.std()['Male'],5)
print('0번 row 결과 확인')
print("전체로 계산 : ", by_total ) 
print("여자 그룹으로 계산 : ", by_female) # transform 결과와 일치
print('\n1번 row 결과 확인')
print("전체로 계산 : ", by_total)
print("남자 그룹으로 계산 : ", by_male) # transform 결과와 일치

0번 row 결과 확인
전체로 계산 :  -1.43699
여자 그룹으로 계산 :  -1.57262

1번 row 결과 확인
전체로 계산 :  -1.43699
남자 그룹으로 계산 :  -1.39656
